In [1]:
#!/bin/python3

import math
import os
import random
import re
import sys

#
# Complete the 'decibinaryNumbers' function below.
#
# The function is expected to return a LONG_INTEGER.
# The function accepts LONG_INTEGER x as parameter.
#

## Initialize Logging

In [2]:
import logging

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)


#Remove existing handlers from root logger
root_logger = logging.getLogger()
root_logger.setLevel(logging.DEBUG)
root_logger.handlers = []

#Create custom logger

logger = logging.getLogger(__name__)
ch = logging.StreamHandler(stream=sys.stdout)
ch.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)

logger.addHandler(ch)

In [3]:
logger.handlers

[<StreamHandler stdout (DEBUG)>]

## Imports

In [4]:
from heapq import heappush, heappop
from collections import defaultdict
from bisect import bisect_left, bisect_right

In [5]:
import math

# Utility

In [6]:
math.log(16,2)

4.0

In [48]:
def generate_start(value):
    
    if value < 10:
        return [value]
        
    else:
        rem = value
        mult = 1
        
        list_digit = []

       #logger.debug.*('in first while')
        while rem >= mult:
           #logger.debug(f'rem = {rem}')
            max_digit = min(8,rem // mult)
            
            list_digit.append(max_digit)
            rem -= max_digit*mult
            
            mult *= 2
            
        #Distribute leftovers
       #logger.debug('in second while')
        while rem > 0:
           #logger.debug(f'rem = {rem}')
            
            index = int(math.log(rem,2))
            list_digit[index] += 1
            rem -= 2**index
        
    return list_digit
    
def get_count(pv, mod_rem, dict_counts):
    
    if dict_counts is None:
        dict_counts = defaultdict(dict)
        dict_counts[0] = {0:[2,4,5,5,5,3,1,0,0], 1:[1,3,5,5,5,4,2,0,0]}
    
    #Accumulate
    if not (pv in dict_counts and mod_rem in dict_counts[pv]):
        list_generate = []
        
        cur_pv = pv
        cur_mod_rem = mod_rem
        
        while cur_mod_rem not in dict_counts[cur_pv]:
            
            list_generate.append((cur_pv, cur_mod_rem))
            
            if cur_mod_rem < 2**cur_pv:
                cur_mode_rem = cur_mod_rem
                
            else:
                cur_mod_rem -= 2**cur_pv
            
            cur_pv -= 1
            
       #logger.debug(list_generate)
        
        #Generate
        for cur_pv, cur_mod_rem in reversed(list_generate):
            
            sum_base = 5**(cur_pv+1)
            even_case = cur_mod_rem < 2**cur_pv
            
            #Generate Even
            if even_case:
                list_input = dict_counts[cur_pv-1][cur_mod_rem]
                list_1 = []
                list_2 = [sum_base]
                for j in range(2, 9, 2):
                    sum_subset = sum(list_input[0:j])
                    list_1.append(sum_subset)
                    list_2.append(sum_base - sum_subset)
                    
#                 list_final = list_1 + list_2[:-1]
                list_final = list_1 + list_2
        
            #Generate Odd
            else:
                list_input = dict_counts[cur_pv-1][cur_mod_rem - 2**cur_pv]
                list_1 = []
                list_2 = [sum_base]
                for j in range(1,8,2):
                    sum_pair = sum(list_input[0:j])
                    list_1.append(sum_pair)
                    list_2.append(sum_base - sum_pair)
                    
                list_final = list_1 + list_2
                    
                    
            dict_counts[cur_pv][cur_mod_rem] = list_final
            
    return dict_counts, dict_counts[pv][mod_rem]
    
    
def generate_index(x, list_index):

    if list_index is None:
        list_index= [(1,2,1, 1),
                     (3,5,2, 2), 
                     (7,11,4, 4), 
                     (15,21,6, 6), 
                     (27,37,10, 10)]
    
    while x >= list_index[-1][0]:
          
        i = len(list_index)

        
        B4 = list_index[(i)//2][3]
        B2 = list_index[(i-2)//2][3]
        
        A = list_index[i-1][2] + B4
        next_count =  A - list_index[i-5][2]

        starting = list_index[-1][1] + list_index[-1][3]
         
        list_index.append((starting, starting+next_count, A, next_count))
        
    return list_index
        
            

In [11]:
math.log(4,2)

2.0

# Func

In [49]:
def decibinaryNumbers(x):
    # Write your code here
    
    if x == 1:
        return 0
    elif x == 2:
        return 1
    
    
    global list_index, dict_counts

    # list_index not initialized
    if 'list_index' not in globals() or list_index == None:
       #logger.info.*(f'list_index NOT in globals')
        
        list_index = generate_index(x, None)
        
#         i_index = len(list_index)-2
    
#    #logger.info.*(f'list_index in globals')

    i_index = bisect_right(list_index, x, key = lambda tup: tup[0]) + 1
    
    if i_index < len(list_index) and x < list_index[i_index][0]:
       #logger.info.*(f'in if')
        i_index -= 2
    # Generate
    else:
       #logger.info.*(f'in else')
        list_index = generate_index(x, list_index)
        i_index = len(list_index)-2

    
   #logger.info.*(f'i_index = {i_index}')
    
    
    value = i_index*2    
    
    if x >= list_index[i_index][1]:
        lower_bound = list_index[i_index][1]
        value += 1 
        
    else:
        lower_bound = list_index[i_index][0]
        
   #logger.info.*(f'value = {value}')
        
   #logger.info.*(f'x = {x}') 
   #logger.info.*(f'lower_bound = {lower_bound}') 
    
    list_digits = [0]* int(math.log(value,2) + 1)
    new_digits = generate_start(value)
    list_digits[0:len(new_digits)] = new_digits
    
   #logger.info.*(f'decib_start = {int("".join(map(str, reversed(list_digits))))}')
    
    
#     while len(list_digits) <= int(math.log(value,2)):
#         list_digits.append(0)
    
    
    rem = x - lower_bound
    
        
   #logger.info.*(f'rem index = {rem}')


    if 'dict_counts' not in globals():
        dict_counts, _ = get_count(0,0,None)
    
    
    rem_val = value
    index = lower_bound
    
    k_break = 0
    
#     max_pv = 
    
    cur_pv = int(math.log((rem_val + 8), 2)) - 5
#     cur_pv = int(math.log((rem + 8), 2)) - 5
    cur_pv = max(0,cur_pv)
    
    while len(list_digits) > 2 and rem > 5 and cur_pv >= 0:
#     while rem >= 5:

#         cur_pv = int(math.log(rem,5)) - 1


        mod_val = 2**(cur_pv+2)
        mod_rem = (rem_val+ 8) % mod_val // 2
        
       #logger.info.*(f'----------------------------------------')
        
       #logger.info.*(f'cur_pv {cur_pv}')
       #logger.info.*(f'mod_val {mod_val}')
       #logger.info.*(f'mod_rem {mod_rem}')
        
#         break
        
        dict_counts, list_counts = get_count(cur_pv, mod_rem, dict_counts)

#         cur_digit = 0 if cur_pv >= len(list_digits)  else list_digits[cur_pv]
    
#         if cur_digit == 0:
#             is_odd = 1 if list_digits[cur_pv-1] % 2 == 1 else 0
            
#         else:
#             is_odd = 1 if cur_digit % 2 == 1 else 0
        
        i_start = 0
        i_end = len(list_counts)
#         quot_val = rem_val // 2**(cur_pv+2) + 1
        quot_val = (rem_val + 8) // 2**(cur_pv+2)
        
       #logger.info.*(f'quot_val {quot_val}')
        
        if quot_val < 8:
            i_start += 8 - quot_val
            
        elif quot_val >= 8 and quot_val < 10:
            pass
            
        elif quot_val == 10:
           #logger.info.*('quot_val == 10')
            if rem_val - (quot_val * mod_val) >= 0:
               #logger.info.*('in i_end - 1')
                i_end -= 1
            
        elif quot_val > 10 and quot_val < 18:
            i_end -= quot_val - 9
#             i_end -= quot_val - 10
        else:
            if quot_val % 2 == 0:
                i_end = 2
            else:
                i_end = 1
                 
            
       #logger.info.*(f'i_start {i_start}')
       #logger.info.*(f'i_end {i_end}')
       #logger.info.*(f'list_counts {list_counts}')
       #logger.info.*(f'list_counts actual {list_counts[i_start:i_end]}')
       #logger.info.*(f'length of counts: {i_end - i_start}')
            
#         break
        
        cumsum = 0
        increase = 0
        for count in list_counts[i_start:i_end]:
            if cumsum + count > rem:
                break
            
            increase +=1
            cumsum += count
            
            
       #logger.info.*(f'cumsum = {cumsum}')
       #logger.info.*(f'increase = {increase}') 
                 
        
#         if increase == 0 or increase == i_end - i_start:
#             print('in equal')
#             break

        
        if increase == 0:
           #logger.info.*('increase == 0')
           #logger.info.*(f'rem_val = {rem_val}')
           #logger.info.*(f'digit = { list_digits[cur_pv+2]}')
            
            rem_val -= 2**(cur_pv+2) * list_digits[cur_pv+2]
            cur_pv -= 1
            
           #logger.info.*(f'rem_val == {rem_val}')
            continue
        
        elif increase == i_end - i_start:
           #logger.info.*('increase == len counts')
           #logger.info.*('increasing pv (first)')
            cur_pv += 1
            
#             if cur_pv > max_pv:
#                 logger.warning(f'cur_pv > max_pv: {cur_pv} > {max_pv}')
#                 break

            continue
#             while list_digits[cur_pv+2] + increase > 9:
#                #logger.info.*('increasing pv')
#                 cur_pv += 1
#             increase = 1

        rem -= cumsum
        index += cumsum
            
        
        list_digits[cur_pv+2] += increase
        list_digits[0:cur_pv+2] = [0] * (cur_pv+2)

        
        rem_val -= 2**(cur_pv+2) * list_digits[cur_pv+2]
         
        
       #logger.info.*(f'rem_val = {rem_val}')
        
        
        new_digits = generate_start(rem_val)
         
       #logger.info.*(f'new_digits = {new_digits}')
        
        list_digits[0:len(new_digits)] = new_digits
        
        cur_pv -= 1
        
       #logger.info.*(f'-------------------')
       #logger.info.*(f'index = {index}')
       #logger.info.*(f'rem index = {rem}')
       #logger.info.*(f'rem value = {rem_val}')
       #logger.info.*(f'list_digits {list_digits}')
       #logger.info.*(f'decibinary = {int("".join(map(str, reversed(list_digits))))}')
        
    
    #Naive implementation?:
    ##Assumes list_digits[1] + rem is always < 10
    ##Assumes list_digits[0] - 2*rem is always >= 0
#     if rem != 0:
# #         print('in rem < 5')
#         list_digits[1] += rem
#         list_digits[0] -= 2*rem
        
    cur_pv = 1
    
   #logger.info.*('in rem < 5')
    
    while rem != 0:
       #logger.info.*(f'in rem == {rem}')
       #logger.info.*(f'list_digits == {list_digits}')
        
#        #logger.info.*(f'cur_pv = {cur_pv}')
#        #logger.info.*(f'cur_pv = {cur_pv}')
        
        
        if list_digits[cur_pv] < 9 and list_digits[cur_pv-1] >= 2:
            
            
            list_digits[cur_pv] += 1
            rem_val = sum([list_digits[i] * 2**i for i in range(cur_pv)]) - 2**cur_pv
            list_digits[0:cur_pv] = [0] * (cur_pv)
            new_digits = generate_start(rem_val)
            list_digits[0:len(new_digits)] = new_digits
            
            rem -= 1
            cur_pv = 1
            
        else:
            cur_pv +=1
                
    
    return int("".join(map(str, reversed(list_digits))))
        


In [30]:
list_digits = [8,8,8,8,2]

In [32]:
list_digits[1+2] += 1

In [33]:
152 - 2**(1+2) * list_digits[1+2]

80

In [ ]:
64 - 2**(2+2) * list_digits[cur_pv+2]

In [19]:
ds[16][19]

(1008, 172)

In [25]:
ds[152][6]

(29688, 145387)

In [19]:
ds[16][25]

(1112, 178)

In [19]:
ds[24][6]

(248, 485)

In [51]:
ds[160][624]

(109888, 172257)

## Res

In [31]:
%%time
# logger.setLevel(logging.INFO)
# logger.setLevel(logging.DEBUG)
logger.setLevel(logging.WARNING)
# target_index = 145387
# target_index = 1593
# target_index = 178
# target_index = 485
# target_index = 186595
n_exp = 18
target_index = 10**n_exp
decibinaryNumbers(target_index)

CPU times: total: 641 ms
Wall time: 642 ms


3112165009320290429

In [28]:
dict_counts

defaultdict(dict,
            {0: {0: [2, 4, 5, 5, 5, 3, 1, 0, 0],
              1: [1, 3, 5, 5, 5, 4, 2, 0, 0]},
             15: {45946: [1152733920,
               33979136733,
               94547993049,
               144111983949,
               152587890625,
               151435156705,
               118608753892,
               58039897576,
               8475906676]},
             14: {13178: [1152733920,
               10318431430,
               22507971383,
               30051278191,
               30517578125,
               29364844205,
               20199146695,
               8009606742,
               466299934]},
             13: {13178: [17491297,
               1135242623,
               3528481819,
               5637215691,
               6103515625,
               6086024328,
               4968273002,
               2575033806,
               466299934]},
             12: {4986: [17491297,
               315181081,
               802570245,
               117

In [32]:
list_index

[(1, 2, 1, 1),
 (3, 5, 2, 2),
 (7, 11, 4, 4),
 (15, 21, 6, 6),
 (27, 37, 10, 10),
 (47, 60, 14, 13),
 (73, 91, 20, 18),
 (109, 131, 26, 22),
 (153, 183, 36, 30),
 (213, 249, 46, 36),
 (285, 330, 59, 45),
 (375, 427, 72, 52),
 (479, 543, 90, 64),
 (607, 679, 108, 72),
 (751, 835, 130, 84),
 (919, 1012, 152, 93),
 (1105, 1215, 182, 110),
 (1325, 1447, 212, 122),
 (1569, 1709, 248, 140),
 (1849, 2003, 284, 154),
 (2157, 2334, 329, 177),
 (2511, 2703, 374, 192),
 (2895, 3109, 426, 214),
 (3323, 3553, 478, 230),
 (3783, 4041, 542, 258),
 (4299, 4576, 606, 277),
 (4853, 5157, 678, 304),
 (5461, 5785, 750, 324),
 (6109, 6465, 834, 356),
 (6821, 7197, 918, 376),
 (7573, 7978, 1011, 405),
 (8383, 8809, 1104, 426),
 (9235, 9699, 1214, 464),
 (10163, 10653, 1324, 490),
 (11143, 11671, 1446, 528),
 (12199, 12756, 1568, 557),
 (13313, 13917, 1708, 604),
 (14521, 15155, 1848, 634),
 (15789, 16467, 2002, 678),
 (17145, 17855, 2156, 710),
 (18565, 19330, 2333, 765),
 (20095, 20897, 2510, 802),
 (21699

In [30]:
dict_counts = None
list_index = None

In [36]:
rem_val = 160
cur_pv = 2
mod_val = 2**(cur_pv+2)
quot_val = (rem_val + 8) // mod_val

In [37]:
quot_val

10

In [42]:
rem_val - (quot_val * mod_val - 8)

8

In [44]:
rem_val - (quot_val * mod_val) > 0

False

In [31]:
(160+8) % mod_val // 2

4

10

In [29]:
160 % 2**(2+2)

0

In [ ]:
160

In [18]:
8 + 16 + 32 + 24 + 32

112

### Res End

## pv = 0

In [26]:
(56 + 8) // 4

16

In [33]:
math.log((24 + 8), 2)

5.0

In [29]:
math.log((56 + 8), 2)

6.0

In [32]:
math.log((120 + 8), 2)

7.0

In [31]:
math.log((248 + 8), 2)

8.0

In [64]:
target_val = 248
int(math.log((target_val + 8), 2)) - 5

3

## pv = 1

In [16]:
(56 + 8) // 8

8

# Test

In [53]:
import pickle

In [54]:
with open('./out/ds.obj', 'rb') as f:
    ds = pickle.load(f)

In [55]:
logger.setLevel(logging.WARNING)

In [56]:
decibinaryNumbers(10)

100

In [ ]:
list_incorrect

## Test all

In [66]:
%%time

logger.setLevel(logging.WARNING)

last_index = 1

list_incorrect = []

temp_limit = 1000000
# temp_limit = 20000

for k, sublist in ds.items():
    for j in range(len(sublist)):
        decib, index = sublist[j]
        
        try: 
            res_decib = decibinaryNumbers(index)

            if decib != res_decib:
                list_incorrect.append((index, f'{decib} != {res_decib}'))
                
        except Exception as e:
            list_incorrect.append((index, e))
        
        last_index += 1
        
#         if last_index >= 0:
#             print(last_index)
            
        if last_index % 10000 == 0:
            print(last_index)
        
        if last_index >= temp_limit:
            break
        
    if last_index >= temp_limit:
        break

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
CPU times: total: 2min 42s
Wall time: 2min 43s


## Test Incorrect only

In [89]:
list_incorrect

[]

In [26]:
[(172257, '109888 != 100000'),
 (175798, '109889 != 100001'),
 (179338, '109898 != 100002'),
 (179339, '109978 != 100010'),
 (182966, '109899 != 100003'),
 (182967, '109979 != 100011'),
 (186592, '109988 != 100004'),
 (186593, '109996 != 100012'),
 (186594, '117988 != 100020'),
 (186595, '117996 != 100100')]

[(172257, '109888 != 100000'),
 (175798, '109889 != 100001'),
 (179338, '109898 != 100002'),
 (179339, '109978 != 100010'),
 (182966, '109899 != 100003'),
 (182967, '109979 != 100011'),
 (186592, '109988 != 100004'),
 (186593, '109996 != 100012'),
 (186594, '117988 != 100020'),
 (186595, '117996 != 100100')]

# Profile

In [15]:
import cProfile

In [16]:
cProfile.run('decibinaryNumbers(target_index)')

         1907 function calls in 0.002 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.002    0.002 1570547015.py:1(decibinaryNumbers)
       19    0.000    0.000    0.000    0.000 1570547015.py:22(<lambda>)
       14    0.000    0.000    0.001    0.000 4009161860.py:1(generate_start)
       18    0.000    0.000    0.000    0.000 4009161860.py:33(get_count)
        1    0.000    0.000    0.000    0.000 4009161860.py:97(generate_index)
        1    0.000    0.000    0.002    0.002 <string>:1(<module>)
      255    0.000    0.000    0.000    0.000 __init__.py:1467(debug)
      346    0.000    0.000    0.000    0.000 __init__.py:1479(info)
      601    0.000    0.000    0.000    0.000 __init__.py:1734(isEnabledFor)
        1    0.000    0.000    0.000    0.000 {built-in method _bisect.bisect_right}
        1    0.000    0.000    0.002    0.002 {built-in method builtins.exec}
        2    0.000  

In [34]:
import random

In [50]:
def random_run():
    target_index = random.randint(1,10**16)
    return decibinaryNumbers(target_index)

In [51]:
def random_batch():
    for i in range(50000):
        random_run()

In [52]:
cProfile.run('random_run()')

         450 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       17    0.000    0.000    0.000    0.000 2611218294.py:1(generate_start)
       16    0.000    0.000    0.000    0.000 2611218294.py:33(get_count)
        1    0.000    0.000    0.001    0.001 3062967072.py:1(decibinaryNumbers)
       17    0.000    0.000    0.000    0.000 3062967072.py:22(<lambda>)
        1    0.000    0.000    0.000    0.000 3062967072.py:246(<listcomp>)
        1    0.000    0.000    0.001    0.001 533731235.py:1(random_run)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 random.py:235(_randbelow_with_getrandbits)
        1    0.000    0.000    0.000    0.000 random.py:284(randrange)
        1    0.000    0.000    0.000    0.000 random.py:358(randint)
        1    0.000    0.000    0.000    0.000 {built-in method _bisect.bisect_right}
        3    0.0

## Batch

In [42]:
logger.setLevel(logging.WARNING)

In [55]:
%%time
random_batch()

CPU times: total: 8.58 s
Wall time: 8.6 s


In [54]:
dict_counts = None
list_index = None
cProfile.run('random_batch()')

         21087098 function calls in 22.155 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   773523    8.863    0.000   12.259    0.000 2611218294.py:1(generate_start)
   775121    1.124    0.000    1.372    0.000 2611218294.py:33(get_count)
       10    0.309    0.031    0.341    0.034 2611218294.py:97(generate_index)
    50000    6.536    0.000   21.757    0.000 3062967072.py:1(decibinaryNumbers)
   857943    0.201    0.000    0.201    0.000 3062967072.py:22(<lambda>)
   104732    0.066    0.000    0.066    0.000 3062967072.py:246(<listcomp>)
        1    0.034    0.034   22.155   22.155 4073945833.py:1(random_batch)
    50000    0.088    0.000   22.121    0.000 533731235.py:1(random_run)
        1    0.000    0.000   22.155   22.155 <string>:1(<module>)
    50000    0.074    0.000    0.108    0.000 random.py:235(_randbelow_with_getrandbits)
    50000    0.104    0.000    0.232    0.000 random.py:284(randrange)
    50000

In [53]:
cProfile.run('random_batch()')

         20173540 function calls in 20.070 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   773002    8.487    0.000   11.692    0.000 2611218294.py:1(generate_start)
   775164    0.659    0.000    0.685    0.000 2611218294.py:33(get_count)
    50000    6.189    0.000   19.712    0.000 3062967072.py:1(decibinaryNumbers)
   858112    0.188    0.000    0.188    0.000 3062967072.py:22(<lambda>)
   104374    0.062    0.000    0.062    0.000 3062967072.py:246(<listcomp>)
        1    0.030    0.030   20.070   20.070 4073945833.py:1(random_batch)
    50000    0.077    0.000   20.040    0.000 533731235.py:1(random_run)
        1    0.000    0.000   20.070   20.070 <string>:1(<module>)
    50000    0.067    0.000    0.098    0.000 random.py:235(_randbelow_with_getrandbits)
    50000    0.097    0.000    0.212    0.000 random.py:284(randrange)
    50000    0.039    0.000    0.251    0.000 random.py:358(randint)
    50000    0.319 

# To do:

1. Implement logic for quot_val 18+
2. Implement test for incorrect only


## Optional
3. Save problematic cases


## 